## 0. Load Data

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input/instacart-market-basket-analysis"]).decode("utf8"))

In [ ]:
import os
import zipfile

for dirname, _, filenames in os.walk('/kaggle/input/instacart-market-basket-analysis/'):
    for filename in filenames:        
        archive = zipfile.ZipFile(os.path.join(dirname, filename), mode='r')
        archive.extractall(path="/kaggle/working")
        archive.close()

print(check_output(["ls", "../working"]).decode("utf8"))

In [ ]:
import pandas as pd

order_products_train = pd.read_csv("../working/order_products__train.csv")
order_products_prior = pd.read_csv("../working/order_products__prior.csv")
orders = pd.read_csv("../working/orders.csv")
products = pd.read_csv("../working/products.csv")
aisles = pd.read_csv("../working/aisles.csv")
departments = pd.read_csv("../working/departments.csv")
sample_submission = pd.read_csv("../working/sample_submission.csv")

print('*** df 생성 완료 ***')

## 1. Basic Exploraty 

In [ ]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = 'warn' 

### orders

In [ ]:
orders.shape

In [ ]:
print(orders.shape)
orders.info()

In [ ]:
import missingno as msno
msno.matrix(orders)

In [ ]:
orders_columns = orders.columns.tolist()

for i in range(0, len(orders_columns)):
    print("***",orders_columns[i],"***")
    print(orders[orders_columns[i]].nunique(),'개')
    print(orders[orders_columns[i]].value_counts(normalize=False, sort=True, dropna=False))

In [ ]:
orders.head()

In [ ]:
orders.eval_set.unique()

In [ ]:
cnt_srs = orders.eval_set.value_counts()
cnt_srs

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Eval set type', fontsize=15)
plt.title('Count of rows in each dataset', fontsize=15)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
def get_unique_count(x):
    return len(np.unique(x))

cnt_srs = orders.groupby("eval_set")["user_id"].aggregate(get_unique_count)
cnt_srs

- 총 고객 수 : 206,209  
- 그 중 131,209의 최근 구매 내역이 test 데이터로 주어짐.  
- 이 데이터를 바탕으로 생성한 모델로 75,000명 주문건의 재주문 여부 예측해야 함.

In [ ]:
orders[orders['eval_set']=='test']

In [ ]:
cnt_srs = orders.groupby("user_id")["order_number"].aggregate(np.max).reset_index()
cnt_srs = cnt_srs.order_number.value_counts()

plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[2])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Maximum order number', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
cnt_srs

- user_id 별로 주문건수가 최소 4건에서 최대 100건인점 확인 가능
- 4건 주문건수를 가진 유저가 가장 높은 비율을 차지하고 있음 확인

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="order_dow", data=orders, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of week', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of order by week day", fontsize=15)
plt.show()

- 0, 1 요일에 주문건이 가장 많은것으로 보아 주말로 추정됨
- 3 요일에 주문건이 가장 적으나 대체적으로 0, 1 을 제외하곤 비슷한 추세를 보임

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="order_hour_of_day", data=orders, color=color[1])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Hour of day', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of order by hour of day", fontsize=15)
plt.show()

- 오전 9시에서 오후 5시 사이에 대부분의 주문이 이루어지고 있음 확인 가능

In [ ]:
grouped_df = orders.groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()
grouped_df = grouped_df.pivot('order_dow', 'order_hour_of_day', 'order_number')

plt.figure(figsize=(12,6))
sns.heatmap(grouped_df, cmap="YlGnBu")
plt.title("Frequency of Day of week Vs Hour of day")
plt.show()

- 0요일 오후 1시\~3시 / 1요일 오전 9시~11시 사이에 가장 주문건이 많이 발생하고 있음 확인 가능

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="days_since_prior_order", data=orders, color=color[3])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Days since prior order', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency distribution by days since prior order", fontsize=15)
plt.show()

In [ ]:
print('nan 값 : ', orders.days_since_prior_order.isnull().sum())
print('nan 비율 : ', round(orders.days_since_prior_order.isnull().sum()/orders.shape[0] * 100), 2)

- 주기를 살펴볼 시, 30일 > 7일에 재구매가 가장 빈번하게 이루어지고 있음을 확인 가능
- 7일에서 30일 사이에 점점 감소하는 추세를 보이나 14일 / 21일 / 28일에 상대적으로 반짝 재구매율이 높아지는 것을 확인 가능
- nan 값은 6%

### order_products_prior

In [ ]:
order_products_prior.shape

In [ ]:
order_products_prior.info()

In [ ]:
order_products_prior.isnull().sum()

In [ ]:
order_products_prior_columns = order_products_prior.columns.tolist()

for i in range(0, len(order_products_prior_columns)):
    print("***",order_products_prior_columns[i],"***")
    print(order_products_prior[order_products_prior_columns[i]].nunique(),'개')
    print(order_products_prior[order_products_prior_columns[i]].value_counts(normalize=False, sort=True, dropna=False))

In [ ]:
order_products_prior.head()

In [ ]:
order_products_prior.reordered.sum() / order_products_prior.shape[0]

- prior 데이터에서 재구매 주문건의 비율은 59%에 달하는것을 확인 가능

In [ ]:
grouped_df = order_products_prior.groupby("order_id")["reordered"].aggregate("sum").reset_index()
grouped_df.loc[grouped_df["reordered"] >= 1] = 1
grouped_df.reordered.value_counts() / grouped_df.shape[0]

- prior 데이터에서 재주문을 하지 않은 고객 비율은 12%임 확인

In [ ]:
grouped_df = order_products_prior.groupby("order_id")["add_to_cart_order"].aggregate("max").reset_index()
cnt_srs = grouped_df.add_to_cart_order.value_counts()
print(cnt_srs)
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of products in the given order', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

- 한 번 구매시 5개의 제품을 구매하는 경우가 228330 조문건으로 가장 많음

## order_products_train

In [ ]:
order_products_train.shape

In [ ]:
order_products_train.info()

In [ ]:
order_products_train_columns = order_products_train.columns.tolist()

for i in range(0, len(order_products_train_columns)):
    print("***",order_products_train_columns[i],"***")
    print(order_products_train[order_products_train_columns[i]].nunique(),'개')
    print(order_products_train[order_products_train_columns[i]].value_counts(normalize=False, sort=True, dropna=False))

In [ ]:
order_products_train.head()

In [ ]:
order_products_train.reordered.sum() / order_products_train.shape[0]

- train 데이터에서 재구매 주문건의 비율은 60%에 달하는점 확인 가능 

In [ ]:
grouped_df = order_products_train.groupby("order_id")["reordered"].aggregate("sum").reset_index()
grouped_df.loc[grouped_df["reordered"] >= 1] = 1
grouped_df.reordered.value_counts() / grouped_df.shape[0]

- test 데이터에서 재주문을 하지 않은 고객 비율은 6.5%임 확인

In [ ]:
grouped_df = order_products_train.groupby("order_id")["add_to_cart_order"].aggregate("max").reset_index()
cnt_srs = grouped_df.add_to_cart_order.value_counts()
print(cnt_srs)
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of products in the given order', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

- 한 주문건에 5건의 제품을 구매하는 경우가 8895 주문건으로 가장 높은 비율을 차지하고 있음

## products

In [ ]:
products.shape

In [ ]:
products.info()

In [ ]:
products_columns = products.columns.tolist()

for i in range(0, len(products_columns)):
    print("***",products_columns[i],"***")
    print(products[products_columns[i]].nunique(),'개')
    print(products[products_columns[i]].value_counts(normalize=False, sort=True, dropna=False))

In [ ]:
products.head()

## aisles

In [ ]:
aisles.shape

In [ ]:
aisles.info()

In [ ]:
aisles_columns = aisles.columns.tolist()

for i in range(0, len(aisles_columns)):
    print("***",aisles_columns[i],"***")
    print(aisles[aisles_columns[i]].nunique(),'개')
    print(aisles[aisles_columns[i]].value_counts(normalize=False, sort=True, dropna=False))

In [ ]:
aisles.head()

## departments

In [ ]:
departments.shape

In [ ]:
departments.info()

In [ ]:
departments_columns = departments.columns.tolist()

for i in range(0, len(departments_columns)):
    print("***",departments_columns[i],"***")
    print(departments[departments_columns[i]].nunique(),'개')
    print(departments[departments_columns[i]].value_counts(normalize=False, sort=True, dropna=False))

In [ ]:
departments.head()

## sample_submission

In [ ]:
sample_submission

## 2. DF Merge

### prior + products + aisles + departments

In [ ]:
order_products_prior = pd.merge(order_products_prior, products, on='product_id', how='left')
order_products_prior = pd.merge(order_products_prior, aisles, on='aisle_id', how='left')
order_products_prior = pd.merge(order_products_prior, departments, on='department_id', how='left')
order_products_prior.head()

In [ ]:
order_products_prior.shape

In [ ]:
cnt_srs = order_products_prior['product_name'].value_counts().reset_index().head(20)
cnt_srs.columns = ['product_name', 'frequency_count']
cnt_srs


- 상위 20개 품목의 대부분이 organic 농산물 제품인 점 확인가능

In [ ]:
cnt_srs = order_products_prior['aisle'].value_counts().head(20)
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[5])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Aisle', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

- aisles 컬럼 기준으로 살펴볼 시 fresh fruits / fresh vegetables 이 상위를 차지하고 잇음 확인 가능

In [ ]:
plt.figure(figsize=(10,10))
temp_series = order_products_prior['department'].value_counts()
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series / temp_series.sum())*100))
plt.pie(sizes, labels=labels, 
        autopct='%1.1f%%', startangle=200)
plt.title("Departments distribution", fontsize=15)
plt.show()

- departments 기준으로 살펴볼 시, produce > dairy eggs가 가장 높은 비율을 차지하고 있음 확인 가능

In [ ]:
grouped_df = order_products_prior.groupby(["department"])["reordered"].aggregate("mean").reset_index()
grouped_df

plt.figure(figsize=(12,8))
sns.pointplot(grouped_df['department'].values, grouped_df['reordered'].values, alpha=0.8, color=color[2])
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Department', fontsize=12)
plt.title("Department wise reorder ratio", fontsize=15)
plt.xticks(rotation='vertical')
plt.show()

- departments별 재주문 비율을 살펴보면 dairy eggs가 가장 높은 재주문을 personal care가 제일 낮은 재구매율을 보이고 있음을 확인 가능

In [ ]:
order_products_prior["add_to_cart_order_mod"] = order_products_prior["add_to_cart_order"].copy()
order_products_prior
# order_products_prior.loc[order_products_prior["add_to_cart_order_mod"] > 70] = 70
grouped_df = order_products_prior.groupby(["add_to_cart_order_mod"])["reordered"].aggregate("sum").reset_index()
grouped_df

In [ ]:
plt.figure(figsize=(12,8))
sns.pointplot(grouped_df['add_to_cart_order_mod'].values, grouped_df['reordered'].values, alpha=0.8, color=color[2])
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Add to cart order', fontsize=12)
plt.title("Add to cart order - Reorder ratio", fontsize=15)
plt.xticks(rotation='vertical')
plt.show()

- **장바구니에 담기는 순서가 이를수록 재구매율이 높아진다** 라는 의미있는 인사이트 확인

In [ ]:
order_products_prior = pd.merge(order_products_prior, orders, on='order_id', how='left')
grouped_df = order_products_prior.groupby(["order_dow"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.barplot(grouped_df['order_dow'].values, grouped_df['reordered'].values, alpha=0.8, color=color[3])
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Day of week', fontsize=12)
plt.title("Reorder ratio across day of week", fontsize=15)
plt.xticks(rotation='vertical')
plt.ylim(0.5, 0.7)
plt.show()

- prior 데이터에선 6 요일에 재구매 비율이 가장 높은것으로 확인

In [ ]:
grouped_df = order_products_prior.groupby(["order_hour_of_day"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.barplot(grouped_df['order_hour_of_day'].values, grouped_df['reordered'].values, alpha=0.8, color=color[4])
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Hour of day', fontsize=12)
plt.title("Reorder ratio across hour of day", fontsize=15)
plt.xticks(rotation='vertical')
plt.ylim(0.5, 0.7)
plt.show()

- 오후 3시에 재구매 비율이 가장 높은것으로 확인

In [ ]:
grouped_df = order_products_prior.groupby(["order_dow", "order_hour_of_day"])["reordered"].aggregate("mean").reset_index()
grouped_df = grouped_df.pivot('order_dow', 'order_hour_of_day', 'reordered')

plt.figure(figsize=(12,6))
sns.heatmap(grouped_df, cmap="YlGnBu")
plt.title("Reorder ratio of Day of week Vs Hour of day")
plt.show()

- prior 데이터에선 6 요일 오후 3시에 가장 많은 재주문이 일어남

### test + products + aisles + departments

In [ ]:
order_products_train = pd.merge(order_products_train, orders, on='order_id', how='left')
grouped_df = order_products_train.groupby(["order_dow"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.barplot(grouped_df['order_dow'].values, grouped_df['reordered'].values, alpha=0.8, color=color[3])
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Day of week', fontsize=12)
plt.title("Reorder ratio across day of week", fontsize=15)
plt.xticks(rotation='vertical')
plt.ylim(0.5, 0.7)
plt.show()

In [ ]:
grouped_df = order_products_train.groupby(["order_hour_of_day"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.barplot(grouped_df['order_hour_of_day'].values, grouped_df['reordered'].values, alpha=0.8, color=color[4])
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Hour of day', fontsize=12)
plt.title("Reorder ratio across hour of day", fontsize=15)
plt.xticks(rotation='vertical')
plt.ylim(0.5, 0.7)
plt.show()

In [ ]:
grouped_df = order_products_train.groupby(["order_dow", "order_hour_of_day"])["reordered"].aggregate("mean").reset_index()
grouped_df = grouped_df.pivot('order_dow', 'order_hour_of_day', 'reordered')

plt.figure(figsize=(12,6))
sns.heatmap(grouped_df, cmap="YlGnBu")
plt.title("Reorder ratio of Day of week Vs Hour of day")
plt.show()